Things i need to do:
- data vis
- make note of hyperparameter tuning process (most important part of presentation)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score,train_test_split, cross_val_predict
from sklearn.metrics import mean_squared_error,r2_score, roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier
import itertools as it

In [34]:
data = pd.read_csv('movies_classification.csv', index_col='Unnamed: 0')

In [35]:
#splitting preds and response
y = data.revenue
X = data.drop(['revenue'], axis = 1)

In [36]:
# splitting test/train data, stratifying response because way more 0 than 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 45)

In [37]:
# making base model to find the range of hyperparameters
model = DecisionTreeClassifier(random_state = 1)
model.fit(X_train, y_train)

ValueError: could not convert string to float: "[{'name': 'Fox Star Studios', 'id': 12154}, {'name': 'White Bucket Productions', 'id': 50921}, {'name': 'Sunland Cinemas', 'id': 50922}]"

In [33]:
X_train.original_language.value_counts().head(10)


en    23919
fr     1796
it     1148
ja     1012
de      791
es      758
ru      623
hi      378
ko      319
zh      305
Name: original_language, dtype: int64